<a href="https://colab.research.google.com/github/Affanamin/Deep-Learning/blob/master/Optimization_using_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [0]:
import tensorflow as tf
from  tensorflow import keras
import numpy as np


In [6]:
print(tf.__version__)

1.15.0


In [0]:
fashion_mnist=keras.datasets.fashion_mnist

In [0]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [0]:
train_images=train_images/255.0
test_images=test_images/255.0

In [16]:
train_images[0].shape

(28, 28)

In [0]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [22]:
tuner_search=RandomSearch(build_model,
                          objective='val_loss',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [23]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 7s 132us/sample - loss: 0.3773 - acc: 0.8647 - val_loss: 0.2793 - val_acc: 0.8960
Epoch 2/3
54000/54000 [==============================] - 7s 131us/sample - loss: 0.2369 - acc: 0.9126 - val_loss: 0.2596 - val_acc: 0.9098
Epoch 3/3
54000/54000 [==============================] - 7s 130us/sample - loss: 0.1787 - acc: 0.9323 - val_loss: 0.2488 - val_acc: 0.9115


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 7s 124us/sample - loss: 0.4066 - acc: 0.8537 - val_loss: 0.3052 - val_acc: 0.8912
Epoch 2/3
54000/54000 [==============================] - 6s 117us/sample - loss: 0.2618 - acc: 0.9036 - val_loss: 0.2622 - val_acc: 0.9067
Epoch 3/3
54000/54000 [==============================] - 6s 119us/sample - loss: 0.2061 - acc: 0.9248 - val_loss: 0.2559 - val_acc: 0.9070


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 7s 123us/sample - loss: 0.3993 - acc: 0.8587 - val_loss: 0.3012 - val_acc: 0.8885
Epoch 2/3
54000/54000 [==============================] - 7s 120us/sample - loss: 0.2535 - acc: 0.9072 - val_loss: 0.2626 - val_acc: 0.9043
Epoch 3/3
54000/54000 [==============================] - 7s 121us/sample - loss: 0.1979 - acc: 0.9264 - val_loss: 0.2435 - val_acc: 0.9135


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 6s 119us/sample - loss: 0.5373 - acc: 0.8102 - val_loss: 0.4516 - val_acc: 0.8362
Epoch 2/3
54000/54000 [==============================] - 6s 116us/sample - loss: 0.3935 - acc: 0.8553 - val_loss: 0.4029 - val_acc: 0.8485
Epoch 3/3
54000/54000 [==============================] - 7s 121us/sample - loss: 0.3541 - acc: 0.8673 - val_loss: 0.3799 - val_acc: 0.8643


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 8s 143us/sample - loss: 0.4682 - acc: 0.8334 - val_loss: 0.3693 - val_acc: 0.8678
Epoch 2/3
54000/54000 [==============================] - 7s 137us/sample - loss: 0.3312 - acc: 0.8778 - val_loss: 0.3225 - val_acc: 0.8830
Epoch 3/3
54000/54000 [==============================] - 7s 136us/sample - loss: 0.2945 - acc: 0.8908 - val_loss: 0.3466 - val_acc: 0.8777


INFO:tensorflow:Oracle triggered exit


In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1115184   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,193,162
Trainable params: 1,193,162
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10
54000/54000 [==============================] - 7s 122us/sample - loss: 0.1508 - acc: 0.9436 - val_loss: 0.2583 - val_acc: 0.9138
Epoch 5/10
54000/54000 [==============================] - 6s 120us/sample - loss: 0.1116 - acc: 0.9586 - val_loss: 0.3032 - val_acc: 0.9078
Epoch 6/10
54000/54000 [==============================] - 7s 121us/sample - loss: 0.0838 - acc: 0.9697 - val_loss: 0.3046 - val_acc: 0.9117
Epoch 7/10
54000/54000 [==============================] - 6s 120us/sample - loss: 0.0626 - acc: 0.9775 - val_loss: 0.3625 - val_acc: 0.9115
Epoch 8/10
54000/54000 [==============================] - 6s 119us/sample - loss: 0.0462 - acc: 0.9828 - val_loss: 0.3934 - val_acc: 0.9148
Epoch 9/10
54000/54000 [==============================] - 7s 121us/sample - loss: 0.0382 - acc: 0.9864 - val_loss: 0.4221 - val_acc: 0.9090
Epoch 10/10
54000/54000 [==============================] - 6s 119us/sample - loss: 0.0320 - acc: 0.9886 - val_l